In [55]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [56]:
pip install -U matplotlib

Requirement already up-to-date: matplotlib in ./opt/anaconda3/lib/python3.8/site-packages (3.3.3)
Note: you may need to restart the kernel to use updated packages.


In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as si
import yfinance as yf
import os

In [58]:
Apple = yf.download("AAPL", start="2019-10-31", end="2020-10-31")

[*********************100%***********************]  1 of 1 completed


In [59]:
Apple['Adj Close'].tail()

Date
2020-10-26    114.851852
2020-10-27    116.399178
2020-10-28    111.008476
2020-10-29    115.121384
2020-10-30    108.672516
Name: Adj Close, dtype: float64

In [60]:
S0 = 108.67              # spot stock price
K = 110.0               # strike
T = 1.0                 # maturity 
r = 0.09                 # risk free rate 
sig = 0.46               # diffusion coefficient or volatility
N = 3                   # number of periods or number of time steps  
payoff = "call"          # payoff 

In [61]:
dT = float(T) / N                             # Delta t
u = np.exp(sig * np.sqrt(dT))                 # up factor
d = 1.0 / u                                   # down factor 

In [62]:
Sbino = np.zeros((N + 1, N + 1))
S[0, 0] = S0
z = 1
for t in range(1, N + 1):
    for i in range(z):
        S[i, t] = S[i, t-1] * u
        S[i+1, t] = S[i, t-1] * d
    z += 1

In [63]:
Sbino

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [64]:
a = np.exp(r * dT)    # risk free compound return
p = (a - d)/ (u - d)  # risk neutral up probability
q = 1.0 - p           # risk neutral down probability
p

0.49065938793361574

In [65]:
S_T = S[:,-1]
V = np.zeros((N + 1, N + 1))
if payoff =="call":
    V[:,-1] = np.maximum(S_T-K, 0.0)
elif payoff =="put":
    V[:,-1] = np.maximum(K-S_T, 0.0)
V

array([[  0.        ,   0.        ,   0.        , 131.06319971],
       [  0.        ,   0.        ,   0.        ,  31.72618067],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ]])

In [66]:
# for European Option
for j in range(N-1, -1, -1):
    for i in range(j+1):
        V[i,j] = np.exp(-r*dT) * (p * V[i,j + 1] + q * V[i + 1,j + 1])
V

array([[ 24.81576731,  44.64960145,  78.08866765, 131.06319971],
       [  0.        ,   7.19317042,  15.10668145,  31.72618067],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ]])

In [67]:
print('European ' + payoff, str( V[0,0]))
#Binomial tree outcome is that EURO call option for AAPL
#Priced at $24.832 with 1 year maturity and Strike Price of $110.

European call 24.81576731261045
